<a href="https://colab.research.google.com/github/Tonnonssi/MNISTCUSTOM-CNN-Classifier/blob/master/CNN_with_MNIST_and_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00 Errors

### ***errors by different dimension***

1.   **images different dimemsion**
#####error message : RuntimeError: stack expects each tensor to be equal size, but got [1, 28, 28] at entry 0 and [28, 1, 28] at entry 3
#####collected data.shape = (1,28,28)
#####augmentated data.shape = (28,28)
#####*solution* : if shape == (28,28) -> image = np.expand_dims(image, axis=0)
\
2.   **np to tensor channel error**
#####error message : RuntimeError: Given groups=1, weight of size [10, 1, 5, 5], expected input[4, 28, 1, 28] to have 1 channels, but got 28 channels instead
#####Knowing that pytorch's input images should be (C x H x W), I made data's dim (1,28,28). But error came out that the dims are (28,1,28). 
#####problem situation: transform function ToTensor() transfer (H x W x C)PIL images or np arr into (C x H x W)tensor. 
#####*solution* : make dim (1,28,28) into (28,28,1) -> img = np.transpose(image_new, (1, 2, 0))
\
### ***errors by data type***

3.   **data type**
#####error message : Input type (double) and bias type (float) should be the same
#####problem situation : input image type = float64
#####*solution* : astype(np.float32)

\
### ***errors by mistakes***


4.   **check input data len**
#####problem situation : after solve error1,(28,28)size image still left. This problem spent a lot of time actually, but the solution was simple. 
#####*solution* : else: 
#####If I had checked the total number of data that went into the dataset from the beginning, I would have quickly noticed that the (28, 28) data was being duplicated. This is because the number of remaining (28, 28) data was the same as the number of augmented data. However, I failed to notice this fact because I did not check it. As a result, I wasted a lot of time and did something wrong.
#####처음부터 들어간 데이터 총 개수를 확인했으면 (28,28)데이터가 중복으로 들어가고 있다는 사실을 금방 알아차릴 수 있었을 것이다. 계속 남아있는 (28,28)데이터의 개수가 증강된 데이터의 개수와 똑같았기 때문이다. 하지만 이걸 확인을 안해서 이상한 짓만 엄청하고 시간을 잡아먹었다. 

\
5.   **Avoid copying the same code block multiple times**
#####problem situation: After verifying that the data was loaded correctly, I added the data loading part to the CollectedDataSet class. However, I made the mistake of copying the data loading function to the cell above the class without modifying it, which caused an issue.
#####데이터 로드가 제대로 되는지를 확인한 다음에 CollectedDataSet 클래스에 데이터 로드 부분을 추가했다. 이때 데이터 로드에서 사용한 함수를 복사해 클래스 앞 셀에다가 배치해두었는데, 그 버전은 수정하지 않아서 문제가 발생했다. 
#####*solution* : When copying the same code multiple times, it is essential to clearly indicate that it has been copied. Otherwise, it can be difficult to identify and locate the problem if one occurs, and it may not be immediately apparent. In particular, when using a class, it is better to include all the functions used inside the class. If a problem occurs, you can resolve it within the class, making it easier to locate and fix errors.
#####같은 코드를 여러 개 복사해서 사용할 때는 꼭 복사했다는 사실을 명시해두어야 한다. 그렇지 않으면 문제 발생시 찾기가 어렵고고 생각이 안난다. 특히, 클래스를 이용한다면 클래스 안에 사용하는 함수를 전부 넣어두는게 낫다. 문제가 발생하더라도 클래스 내부에서만 문제를 해결하면 되기 때문에 오류를 찾기 용이하다.  



# 01 Intro

이 주피터 노트북에는 CNN(합성곱 신경망)을 이용하여 MNIST, Custom Dataset을 학습시키는 코드가 기술되어 있다. 추가적으로 내가 커스텀 데이터를 만지면서 생겼던 여러 문제와 해결방법이 들어 있다. 

> **Summary**
1. MNIST DataSet
2. Custom DataSet
3. Net
4. Trainer
5. Execute



# 02 DataSet (MNIST,CUSTOM)

## 01 Preparing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.optim as optim
import time
import pandas as pd

uint8
float64

## 02 Custom Dataset class

In [ ]:
class CollectedDataset(Dataset):
    '''
    custom data can be divided into two types. 

    1. collected data : data collected by drawing nums. 
                        its arr has only 0,1 values 

    2. augmentated data : data augmentated by turning, expanding, or both. 
                          its arr has (0-1) values or (0-255: Astype uint8)

    Since I saved them divided, I have to load them seperated. 

    collected data and augmentated data are stored in self.data_list
    
    '''
    def __init__(self, collected_url, augmentated_url, transform=None):

        self.collected_data = self.download_files(collected_url)
        self.augmentated_data = self.download_files(augmentated_url)

        self.data_list = self.collected_data + self.augmentated_data

        self.transform = transform

    def download_files(self, data_url:str) -> list:
        data_lst = []
        fnames = self.make_fname_lst(data_url)
        for fname in fnames:
            data = self.open_pickle_data(data_url+fname)
            tu_data = self.select_answerimage(data)
            data_lst.extend(tu_data)

        return data_lst

    def make_fname_lst(self, pre_url:str) -> list:
        fname_lst = os.listdir(pre_url)
        return fname_lst

    def open_pickle_data(self, url: str) -> list:
        with open(url, 'rb') as f:
            while True:
                try:
                    data = pickle.load(f)
                except EOFError:
                    break
        return data

    def select_answerimage(self, lst2d): 
        lst = []
        for data in lst2d:
            answer = data[0]
            img = data[2]
            
            if img.dtype == 'float64':
                img = (img *255).astype(np.uint8) #주의!
        
            if img.shape == (28, 28):
                image_new = np.expand_dims(img, axis=0) #sol1
                img = np.transpose(image_new, (1, 2, 0)) #sol2
                lst.append((img, answer))
            else:
                img= np.transpose(img, (1, 2, 0)) #sol2
                lst.append((img, answer))
            
        return lst

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        # img = self.data_list[idx][0].astype(np.float32) #sol2
        img = self.data_list[idx][0]
        label = self.data_list[idx][1]

        if self.transform:
            img = self.transform(img)
 
        return (img, label)

## 03 MNIST CUSTOM trainset

In [ ]:
MNIST_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform= transforms.ToTensor()) #return : (image, target)
CUSTOM_trainset = CollectedDataset(collected_url = '/content/drive/MyDrive/MNISTandCustom/datafile/collected_data/',
                            augmentated_url = '/content/drive/MyDrive/MNISTandCustom/datafile/augmentation/',
                            transform=transforms.ToTensor()) #return : (image, target)

# 03 Net

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.pooling import MaxPool2d

In [ ]:
class Net(nn.Module):
    def __init__(self, bn=False):
        super().__init__()

        self.bn = bn

        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,5)

        self.pool = MaxPool2d(2,2)

        self.fc1 = nn.Linear(20*4*4, 100)
        self.fc2 = nn.Linear(100,10)
        #batch normalization
        self.bn1 = nn.BatchNorm2d(10)
        self.bn2 = nn.BatchNorm2d(20)

    def forward(self,x):
        if self.bn == True:
            x = self.pool(F.relu(self.bn1(self.conv1(x))))
            x = self.pool(F.relu(self.bn2(self.conv2(x))))
        else:
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) # flat 꼭 해요오오
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

# Trainer class

In [ ]:
class Trainer:
    def __init__(self, fname, trainloader, testloader, epochs, net):
        self.fname = fname 

        self.trainloader = trainloader
        self.testloader = testloader

        self.epochs = epochs

        self.net = net

        self.criterian = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.net.parameters(), lr=0.001, momentum= 0.9)

        self.process_txt = ''

        self.bundle = 100

    def train(self, dataloader, model, criterian, optimizer, t):

        start = time.time()

        running_loss = 0.0

        for batch, data in enumerate(dataloader, 1):
            inputs, labels = data

            optimizer.zero_grad()

            outputs  = model(inputs)
            loss = criterian(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if batch % self.bundle == 0:
                txt = f"[{t+1}, {batch}] loss: {running_loss /2000:.3f} --------------- {time.time()-start:.3f}\n"
                self.process_txt += txt
                print(txt)
                running_loss = 0.0 # 졸라라 애먹음  

    def test(self, dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0.0, 0.0
        with torch.no_grad():
            for inputs, labels in dataloader:
                pred = model(inputs)
                test_loss += loss_fn(pred, labels).item()
                correct += (pred.argmax(1) == labels).type(torch.float).sum().item()

            test_loss /= num_batches
            correct /= size
            txt = f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
            self.process_txt += txt
            print(txt)

        self.correct = correct
        self.output_fname = f'{self.fname}_{(100*self.correct):>0.1f}%'

    def test_visualize(self, net=None, dataloader=None):
        if dataloader: 
            datas, labels = next(iter(dataloader))
        else:
            datas, labels = next(iter(self.trainloader)) 

        if net:
            net.eval()
            pred = net(datas)
        else:
            self.net.eval()
            pred = self.net(datas)

        np_label = np.expand_dims(labels.numpy(), axis=0)
        np_pred = np.expand_dims(pred.argmax(1).numpy(), axis=0)

        total_len = len(labels)
        arr = np.empty((0,total_len),int)
        arr = np.append(arr, np.array(nlabel), axis=0)
        arr = np.append(arr, np.array(npred), axis=0)

        df = pd.DataFrame(arr, index = ['input', 'prediced'])
        styled_df = df.style.apply(highlight_row, axis=1)

        print(f'''
        input labels are : {labels}
        predicted labels are : {pred.argmax(1)}
        ''')
        
    def highlight_row(self, row):
        print(row['Name'])
        if row['Name'] == 'Bob':
            return ['background-color: yellow']
        else:
            return ['background-color: white']
    


    def process(self):
        self.process_txt += f'Epochs : {self.epochs}\n'

        for t in range(self.epochs):
            txt = f"Epochs {t+1}\n-------------------------------\n"
            self.process_txt += txt
            print(txt)
            self.train(self.trainloader, self.net, self.criterian, self.optimizer,t)
            self.test(self.testloader, self.net, self.criterian)
        print('Done!')
        # self.download_state()

In [ ]:
class TestModel:
    def __init__(self):
        

# Save Model 

In [ ]:
class SaveModel:
    def __init__(self, output_fname, net_structure, net, process_txt=None):
        self.output_fname = output_fname
        self.net_structure = net_structure
        self.net = net
        self.process_txt = process_txt

        self.save_process()
        self.save_datafile()
        self.save_NetClass()
        self.save_state()
        self.save_model()

    def save_process(self):
        fname = f'{self.output_fname}_process.txt'
        url = '/content/drive/MyDrive/MNISTandCustom/trained_model/process/'+fname
        f = open(url, 'w')
        f.write(self.process_txt)
        f.close()

    def save_datafile(self):
        fname = f'{self.output_fname}__data.txt'
        url = '/content/drive/MyDrive/MNISTandCustom/trained_model/data/'+fname
        f = open(url, 'w')
        f.close()

    def save_NetClass(self):
        fname = f'{self.output_fname}__class.pkl'
        url = '/content/drive/MyDrive/MNISTandCustom/trained_model/NetClass/'+fname
        with open(url, 'wb') as f:
            pickle.dump(self.net_structure, f)
            f.close()       
        print(f"Saved PyTorch Model NetClass to '{fname}'")


    def save_state(self):
        fname = f'{self.output_fname}.pth'
        torch.save(self.net.state_dict(),'/content/drive/MyDrive/MNISTandCustom/trained_model/state/'+fname)
        print(f"Saved PyTorch Model State to '{fname}'")

    def save_model(self):
        fname = f'{self.output_fname}.pt'        
        torch.save(self.net, '/content/drive/MyDrive/MNISTandCustom/trained_model/model/'+fname)
        print(f"saved PyTorch Model to '{fname}'")

# Execute

### MNIST testset

In [ ]:
import torchvision
MNIST_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True,transform= transforms.ToTensor())
MNIST_testloader = torch.utils.data.DataLoader(MNIST_testset, batch_size=batch_size, shuffle=True, num_workers=2)

### 01 train, test : same, batch normalization = True

In [ ]:
batch_size = 32
trainset = MNIST_trainset + CUSTOM_trainset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
def save_NetClass():
    fname = f'class.pkl'
    url = '/content/drive/MyDrive/MNISTandCustom/trained_model/NetClass/'+fname
    with open(url, 'wb') as f:
        pickle.dump(Net(bn=True), f)
        f.close()       
    print(f"Saved PyTorch Model NetClass to '{fname}'")

In [ ]:
save_NetClass()

Saved PyTorch Model NetClass to 'class.pkl'


In [ ]:
net = Net(bn=True)

trainer = Trainer(fname='with_bn_model,MNIST+CUSTOM',
                  trainloader=trainloader, 
                  testloader=MNIST_testloader,
                  epochs=15,
                  net=net)

trainer.bundle = 1000

trainer.process()

saveModel = SaveModel(output_fname=trainer.output_fname, 
                      net_structure=net,
                      net=trainer.net,
                      process_txt=trainer.process_txt)

Epochs 1
 -------------------------------

[1, 1000] loss: 0.527 --------------- 14.534

[1, 2000] loss: 0.304 --------------- 28.543

Test Error : 
 Accuracy : 86.1%, Avg loss: 0.358520 

Epochs 2
 -------------------------------

[2, 1000] loss: 0.148 --------------- 13.510

[2, 2000] loss: 0.109 --------------- 29.570

Test Error : 
 Accuracy : 98.1%, Avg loss: 0.058198 

Epochs 3
 -------------------------------

[3, 1000] loss: 0.081 --------------- 13.626

[3, 2000] loss: 0.074 --------------- 26.952

Test Error : 
 Accuracy : 98.3%, Avg loss: 0.057774 

Epochs 4
 -------------------------------

[4, 1000] loss: 0.061 --------------- 15.798

[4, 2000] loss: 0.054 --------------- 29.111

Test Error : 
 Accuracy : 98.5%, Avg loss: 0.046668 

Epochs 5
 -------------------------------

[5, 1000] loss: 0.047 --------------- 12.912

[5, 2000] loss: 0.044 --------------- 25.505

Test Error : 
 Accuracy : 98.5%, Avg loss: 0.052254 

Epochs 6
 -------------------------------

[6, 1000] lo

In [ ]:
net = Net(bn=True)

trainer = Trainer(fname='with_bn_model',
                  trainloader=trainloader, 
                  testloader=trainloader,
                  epochs=30,
                  net=net)

trainer.process()

saveModel = SaveModel(output_fname=trainer.output_fname, 
                      net_structure=net,
                      net=trainer.net,
                      process_txt=trainer.process_txt)
# trainer.process()

# trainer.save_process()
# trainer.save_datafile()
# trainer.save_model()
# trainer.save_state()
# trainer.save_NetClass()

Epochs 1
 -------------------------------

[1, 100] loss: 0.111 --------------- 0.976

[1, 200] loss: 0.101 --------------- 1.853

[1, 300] loss: 0.093 --------------- 2.759

[1, 400] loss: 0.084 --------------- 3.669

[1, 500] loss: 0.074 --------------- 4.616

Test Error : 
 Accuracy : 59.4%, Avg loss: 1.334734 

Epochs 2
 -------------------------------

[2, 100] loss: 0.061 --------------- 1.501

[2, 200] loss: 0.051 --------------- 2.764

[2, 300] loss: 0.045 --------------- 3.886

[2, 400] loss: 0.041 --------------- 5.034

[2, 500] loss: 0.036 --------------- 5.844

Test Error : 
 Accuracy : 79.9%, Avg loss: 0.615098 

Epochs 3
 -------------------------------

[3, 100] loss: 0.030 --------------- 0.871

[3, 200] loss: 0.026 --------------- 1.677

[3, 300] loss: 0.025 --------------- 2.498

[3, 400] loss: 0.022 --------------- 3.322

[3, 500] loss: 0.021 --------------- 4.150

Test Error : 
 Accuracy : 87.9%, Avg loss: 0.367377 

Epochs 4
 -------------------------------

[4, 10

In [ ]:
net.test_visualize(net=net, dataloader=None)

In [ ]:
with open('/content/drive/MyDrive/MNISTandCustom/trained_model/NetClass/with_bn_model_63.5%__class.pkl', 'rb') as f:
    model = pickle.load(f)
    print('debug')
    f.close()

debug


In [ ]:
trainer.test_visualize()


        input labels are : tensor([4, 2, 0, 0, 7, 5, 6, 3, 9, 4, 4, 7, 8, 1, 8, 7, 9, 0, 8, 0, 1, 5, 6, 5,
        7, 3, 3, 1, 4, 8, 8, 4])
        predicted labels are : tensor([4, 2, 0, 0, 7, 5, 6, 3, 9, 4, 4, 7, 8, 1, 8, 7, 9, 0, 8, 0, 1, 5, 6, 5,
        7, 3, 3, 1, 4, 8, 8, 4])
        


In [ ]:
trainer = Trainer(fname='without_bn_model',
                  trainloader=trainloader, 
                  testloader=trainloader,
                  epochs=20,
                  net=Net(bn=True))
trainer.process()
trainer.download_state()

### 02 train : custom, test : MNIST , batch normalization = False

In [ ]:
trainer = Trainer(fname='without_bn_model',
                  trainloader=trainloader, 
                  testloader=testloader,
                  epochs=20,
                  net=Net())
trainer.process()
trainer.save_process()
trainer.save_datafile()
trainer.save_model()

In [ ]:
net = trainer.net

fname = f'{trainer.fname}_{(100*trainer.correct):>0.1f}%.pth'
torch.save(net.state_dict(),fname)
print(f"Saved PyTorch Model State to '{fname}'")

Saved PyTorch Model State to 'with_bn_model_98.9%.pth'


- test data : custom, batch normalization = False

## 03 train : collected + turn data , test : MNIST, batch normalization = True


In [ ]:
trainer = Trainer(fname='with_bn_model_only_turnandcollect',
                  trainloader=trainloader, 
                  testloader=testloader,
                  epochs=30,
                  net=Net(bn=True))
trainer.process()
trainer.download_state()

Epochs 1
 -------------------------------
[1, 100] loss: 0.113 --------------- 2.039
[1, 200] loss: 0.109 --------------- 3.658
Test Error : 
 Accuracy : 38.4%, Avg loss: 2.010480 

Epochs 2
 -------------------------------
[2, 100] loss: 0.092 --------------- 1.101
[2, 200] loss: 0.081 --------------- 2.133
Test Error : 
 Accuracy : 54.9%, Avg loss: 1.453178 

Epochs 3
 -------------------------------
[3, 100] loss: 0.071 --------------- 1.101
[3, 200] loss: 0.064 --------------- 2.135
Test Error : 
 Accuracy : 63.6%, Avg loss: 1.177090 

Epochs 4
 -------------------------------
[4, 100] loss: 0.058 --------------- 1.744
[4, 200] loss: 0.053 --------------- 3.378
Test Error : 
 Accuracy : 69.1%, Avg loss: 0.954091 

Epochs 5
 -------------------------------
[5, 100] loss: 0.047 --------------- 1.084
[5, 200] loss: 0.047 --------------- 2.117
Test Error : 
 Accuracy : 69.3%, Avg loss: 0.905806 

Epochs 6
 -------------------------------
[6, 100] loss: 0.042 --------------- 1.117
[6, 2

## 04 train : custom, test : MNIST , batch normalization = True

In [ ]:
trainer = Trainer(fname='with_bn_model',
                  trainloader=trainloader, 
                  testloader=testloader,
                  epochs=20,
                  net=Net(bn=True))
trainer.process()
trainer.save_process()
trainer.save_datafile()
trainer.save_model()

Epochs 1
 -------------------------------

[1, 100] loss: 0.113 --------------- 1.268

[1, 200] loss: 0.104 --------------- 2.390

[1, 300] loss: 0.096 --------------- 3.521

[1, 400] loss: 0.089 --------------- 4.693

[1, 500] loss: 0.081 --------------- 5.861

Test Error : 
 Accuracy : 39.6%, Avg loss: 1.737087 

Epochs 2
 -------------------------------

[2, 100] loss: 0.070 --------------- 1.182

[2, 200] loss: 0.060 --------------- 2.243

[2, 300] loss: 0.055 --------------- 3.298

[2, 400] loss: 0.048 --------------- 4.383

[2, 500] loss: 0.045 --------------- 5.483

Test Error : 
 Accuracy : 54.1%, Avg loss: 1.712085 

Epochs 3
 -------------------------------

[3, 100] loss: 0.037 --------------- 1.102

[3, 200] loss: 0.037 --------------- 2.631

[3, 300] loss: 0.035 --------------- 4.380

[3, 400] loss: 0.034 --------------- 5.783

[3, 500] loss: 0.033 --------------- 6.935

Test Error : 
 Accuracy : 59.2%, Avg loss: 1.673493 

Epochs 4
 -------------------------------

[4, 10

# temp

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'Score': [85, 90, 95, 80]}
df = pd.DataFrame(data)

# Define a function to apply styles to specific rows
def highlight_row(row):
    print(row['Name'])
    if row['Name'] == 'Bob':
        return ['background-color: yellow']
    else:
        return ['background-color: white']

# Apply the styles to the DataFrame
styled_df = df.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_df

In [ ]:
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'Score': [25, 25, 95, 80]}
df1 = pd.DataFrame(data)
df1

,Name,Age,Score
0,Alice,25,25
1,Bob,30,25
2,Charlie,35,95
3,David,40,80


In [ ]:
import numpy as np

In [ ]:
def highlight_df1(x, color):
    if x['Name'] == 'Alice':
        return np.where(x == 25, f"color: {color};", None)
    # else:
    #     return 적절한 리턴값

In [ ]:
df1.style.apply(highlight_df1, color='orange', axis=1)

,Name,Age,Score
0,Alice,25,25
1,Bob,30,25
2,Charlie,35,95
3,David,40,80


In [ ]:
pred = [1,0,0,0,1,0,0,1,1,1,1,0,1,0]
target = [0,1,0,0,0,1,1,0,0,1,0,0,0,1]

nlabel = np.expand_dims(np.array(pred), axis=0)
npred = np.expand_dims(np.array(target), axis=0)

total_len = len(pred)
arr = np.empty((0,total_len),int)
arr = np.append(arr, np.array(nlabel), axis=0)
arr = np.append(arr, np.array(npred), axis=0)
arr = np.append(arr, nlabel == npred, axis=0)

# data = {'pred': pred, 'Score': target}
df = pd.DataFrame(arr, index=['input', 'prediced', 'TF'])
df = df.reset_index()

In [ ]:
def highlight_1(x, color):
    if x['index'] == 'TF':
        return np.where(x == 1, f"color: {color};", None)

In [ ]:
df.style.apply(highlight_1, color='orange', axis=1)

In [ ]:
row = pd.DataFrame(np.where((nlabel == npred) == 1, True, False))

In [ ]:
import pandas as pd

In [ ]:
datas, labels = next(iter(trainloader))

In [ ]:
trainer.net.eval()
pred = trainer.net(datas)

In [ ]:
labels == pred.argmax(1)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True])

In [ ]:
#텐서 넘파이로 변환 
nlabel = np.expand_dims(labels.numpy(), axis=0)
npred = np.expand_dims(pred.argmax(1).numpy(), axis=0)

In [ ]:
nlabel == npred

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True]])

In [ ]:
def highlight_False(s, props=''):
    return np.where(s == True, props, '')

datas, labels = next(iter(trainloader))

trainer.net.eval()
pred = trainer.net(datas)

#텐서 넘파이로 변환 
nlabel = np.expand_dims(labels.numpy(), axis=0)
npred = np.expand_dims(pred.argmax(1).numpy(), axis=0)

total_len = len(labels)
arr = np.empty((0,total_len),int)
arr = np.append(arr, np.array(nlabel), axis=0)
arr = np.append(arr, np.array(npred), axis=0)

df = pd.DataFrame(arr, index = ['input', 'prediced'])

TF_row = pd.DataFrame(np.where((nlabel == npred) == 1, True, False))
TF_row.loc[TF_row.index.max()+1] = df.iloc[0]
TF_row.loc[TF_row.index.max()+1] = df.iloc[1]

s = TF_row.style
s.apply(highlight_False, props='color:white;background-color:black', axis=0)
s

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
1,6,8,8,8,7,6,7,4,8,0,5,9,0,4,2,2,4,9,4,2,9,1,6,4,7,8,5,4,6,3,3,9
2,6,8,8,8,7,6,7,4,8,0,5,5,0,4,2,2,4,9,4,2,9,1,6,4,7,8,5,4,5,3,3,9


In [ ]:
TF_row = pd.DataFrame(np.where((nlabel == npred) == 1, True, False))
TF_row.loc[TF_row.index.max()+1] = df.iloc[0]
TF_row.loc[TF_row.index.max()+1] = df.iloc[1]
TF_row

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,5,6,3,5,3,7,8,8,2,5,...,6,7,1,7,2,9,7,4,1,7
2,5,6,3,5,3,7,8,8,2,5,...,6,7,1,7,2,9,7,4,1,7


In [ ]:
# 하나의 arr로 합치기
total_len = len(labels)
arr = np.empty((0,total_len),int)
arr = np.append(arr, np.array(nlabel), axis=0)
arr = np.append(arr, np.array(npred), axis=0)
# arr = np.append(arr, np.where((nlabel == npred) == 1, True, False), axis=0)

In [ ]:
df = pd.DataFrame(arr, index = ['input', 'prediced'])
df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
input,5,6,3,5,3,7,8,8,2,5,...,6,7,1,7,2,9,7,4,1,7
prediced,5,6,3,5,3,7,8,8,2,5,...,6,7,1,7,2,9,7,4,1,7


In [ ]:
df.loc[df.index.max()+1] = TF_row.iloc[0]
df

TypeError: ignored

In [ ]:
df.index

Index(['input', 'prediced', 'T/F'], dtype='object')

In [ ]:
help(np.where)

In [ ]:
def highlight_False(s, props=''):
    return np.where(s == 1, props, '')
    
s = df.style
s.apply(highlight_False, props='color:white;background-color:black', axis=0)
s

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
input,2,2,2,8,3,5,4,1,6,2,3,9,0,7,3,0,1,4,4,7,1,3,7,8,0,6,0,6,1,2,9,8
prediced,2,2,2,8,3,3,4,1,6,2,3,9,0,7,3,0,1,4,8,7,1,3,7,8,0,6,0,6,1,2,9,8
T/F,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1


# checking

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
from torchsummary import summary

In [ ]:
model = torch.load('/content/drive/MyDrive/MNISTandCustom/trained_model/model/with_bn_model_63.8%.pth')

In [ ]:
print(model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [ ]:
testset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
summary(model, input_size=(1, 28, 28))

RuntimeError: ignored

# pandas styler

In [ ]:
import pandas as pd
import random
import numpy as np

In [ ]:
n = 10
pred = [random.randint(0, 1) for _ in range(n)]
target = [random.randint(0, 1) for _ in range(n)]
result = np.array(pred) == np.array(target)

In [ ]:
df = pd.DataFrame({'pred':pred, 'target':target, 'result':result})

In [ ]:
def highlight_true(target_col):
    # # print(target_col.name)
    if target_col.name == 'result':
        return ['color:yellow' if v else '' for v in target_col]
    else:
        res = ['' for v in target_col]
        return np.array(res)
    # else:
    #     return['' for v in target_col]

In [ ]:
def test_val(x):
    if x == 3:
        return True

In [ ]:
a = [1,2,3]
b = a

print(id(a), id(b))

139933503027968 139933503027968


In [ ]:
infinite = True
ugly = True

if infinite and ugly:
    print('do something')

condition = None

if condition:
    print('good!')
else:
    print('bad!')

do something
bad!


In [ ]:
bool(0)

False

In [ ]:
import copy
my_dict = {'name':None, 'height':None}
my_list = [copy.deepcopy(my_dict) for _ in range(10)]
my_list[0]['name'] = 'jimin'
my_list

[{'name': 'jimin', 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None},
 {'name': None, 'height': None}]

In [ ]:
df.style.apply(highlight_true, axis=0)

,pred,target,result
0,0,1,False
1,1,0,False
2,0,1,False
3,0,0,True
4,1,1,True
5,1,0,False
6,1,1,True
7,1,0,False
8,0,1,False
9,0,0,True


# original

In [ ]:
s = 'string is good'
s[::-1]
type(s.split(' '))

list

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epochs {t+1}\n -------------------------------")
    train(trainloader, net, criterian, optimizer, t)
    test(testloader,net,criterian)
print('Done!')

In [ ]:
from torchvision.transforms.transforms import ToTensor
import random

data = CollectedDataset(collected_url = '/content/drive/MyDrive/data_file/collected_data/',
                            augmentated_url = '/content/drive/MyDrive/data_file/augmentation/')
new = data.data_list
i = random.randrange(0,len(new))
a = new[i][0]

transform = transforms.Compose([transforms.ToTensor()])
tensor = transform(a)
tensor

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,